In [6]:
# Cauchy MonteCarlo
library(tidyverse)

# Parameters:
# func : a function whose first argument is a real vector of parameters
# func returns a log10 of the likelihood function
# theta.init : the initial value of the Markov Chain (and of func)
# n.sample: number of required samples
# sigma : standar deviation of the gaussian MCMC sampling pdf


metropolis.1dim <- function(func , theta.init , n.sample , sigma) {
    theta.cur <- theta.init
    func.Cur <- func(theta.cur)
    func.Samp <- matrix(data=NA, nrow=n.sample , ncol=2+1)
    n.accept <- 0
    rate.accept <- 0.0
    
    for (n in 1:n.sample) {
        theta.prop <- rnorm(n=1, mean = theta.cur, sigma)
        func.Prop <- func(theta.prop)
        logMR <- func.Prop - func.Cur          # Log10 of the Metropolis ratio
        if ( logMR >=0 || logMR >log10(runif(1)) ) {
            theta.cur <- theta.prop
            func.Cur <- func.Prop
            n.accept <- n.accept + 1
        }
        func.Samp[n, 1] <- func.Cur
        func.Samp[n, 2] <- theta.cur
    }
    return(func.Samp)
}

#
# Our test function
#
testfunc <- function(theta) {
    return(dcauchy(theta , -10, 2,) + 4*dcauchy(theta , 10, 4))
}
#
# - interface for the metropolis function , gets the log10 of test function
testfunc.metropolis <- function(theta) {
    return(log10(testfunc(theta)))
}
### Running parameters
theta.init <- -5
sample.sig <- readline("Enter sigma sample: ") %>% as.numeric()  
n.sample <- 10^5
demo <- TRUE
set.seed(20190513)
chain <- metropolis.1dim(func=testfunc.metropolis , theta.init = theta.init , n.sample = n.sample , sigma = sample.sig^2)

#
# Here are the plots
#
par(mfrow=c(2,2), mgp=c(2,0.8,0), mar=c(3.5,3.5,1,1), oma=0.1*c(1,1,1,1))
x <- seq(-50, 50, length.out=10^4)
y <- testfunc(x)
ymax <- 1.05 * max(y)

plot(x, y, ylim=c(0,max(y)*1.10), type="l", lwd=2, col="firebrick3", xlab=expression(theta), 
     ylab=expression(paste("f(",theta ,")", sep="")))
plot(x, y, type="n", yaxs="i", ylim=c(0, 1.05*max(y)), xlab=expression(theta), ylab=expression(paste("f(",theta ,")", sep="")))

sa <- which(chain[,2]>=min(x) & chain[,2]<=max(x))
hist <- hist(chain[sa,2], breaks=seq(from=min(x), to=max(x), length.out=100), plot=FALSE)
Zhist <- sum(hist$counts)*diff(range(hist$breaks))/(length(hist$counts))
Zfunc <- 4.7  # ????

# c(hist$counts*Zfunc/Zhist)

lines(hist$breaks , c(Zfunc*hist$counts/(sum(hist$counts)*1.010101), 0), col="navy", type="s", lwd=2, lty=5)
lines(x, y, col="firebrick3", lwd=1, lty=1)
leg.labels = c("analytical", "MCMC")
leg.ltype = c(1, 5)
leg.colors = c("firebrick3","navy")
legend("topleft", inset=.05, bty="n", legend = leg.labels, lty=leg.ltype, col=leg.colors, lwd = 2)

ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'ellipsis' 0.3.1 is being loaded, but >= 0.3.2 is required


In [ ]:
library(coda)

c.chain1 <- as.mcmc(chain[,2])
my.lags = seq(0,500,10)
y1 <- autocorr(c.chain1, lags=my.lags)
plot(my.lags , y1, ylim=c(0,1), pch=12, col="navy", xlab="lag", ylab="ACF", cex=1.3)
text(400,0.9, paste("sigma=", sample.sig))
text(400,0.85, sprintf("effective size: %.2f", effectiveSize(c.chain1)))

In [ ]:
# Cauchy MonteCarlo


# Parameters:
# func : a function whose first argument is a real vector of parameters
# func returns a log10 of the likelihood function
# theta.init : the initial value of the Markov Chain (and of func)
# n.sample: number of required samples
# sigma : standar deviation of the gaussian MCMC sampling pdf
library(tidyverse)

metropolis.1dim <- function(func , theta.init , n.sample , sigma) {
    theta.cur <- theta.init
    func.Cur <- func(theta.cur)
    func.Samp <- matrix(data=NA, nrow=n.sample , ncol=2+1)
    n.accept <- 0
    rate.accept <- 0.0
    
    for (n in 1:n.sample) {
        theta.prop <- rnorm(n=1, mean = theta.cur, sigma)
        func.Prop <- func(theta.prop)
        logMR <- func.Prop - func.Cur          # Log10 of the Metropolis ratio
        if ( logMR >=0 || logMR >log10(runif(1)) ) {
            theta.cur <- theta.prop
            func.Cur <- func.Prop
            n.accept <- n.accept + 1
        }
        func.Samp[n, 1] <- func.Cur
        func.Samp[n, 2] <- theta.cur
    }
    return(n.accept)
}

#
# Our test function
#
testfunc <- function(theta) {
    return(dcauchy(theta , -10, 2,) + 4*dcauchy(theta , 10, 4))
}
#
# - interface for the metropolis function , gets the log10 of test function
testfunc.metropolis <- function(theta) {
    return(log10(testfunc(theta)))
}
### Running parameters
theta.init <- -5
sample.sig <- readline("Enter sigma sample: ") %>% as.numeric()   
n.sample <- 10^5
demo <- TRUE
set.seed(20190513)


Acc.step <- metropolis.1dim(func=testfunc.metropolis , theta.init = theta.init , n.sample = n.sample , sigma = sample.sig^2)

cat("\nNumbers of accepted step: ", Acc.step)
cat("\nAcceptance rate: ", Acc.step*100/n.sample, "%")